# Deciphering Code with Character-Level RNN

In this notebook, we'll look at how to build a recurrent neural network and train it to decipher strings encrypted with a certain cipher.

This exercise will make you familiar with the techniques of preprocessing and model-building that will come in handy when you start building more advanced models for machine translation, text summarization, and beyond.

## Dataset
The dataset we have consists of 10,000 encrypted phrases and the plaintext version of each encrypted phrase.

Let's start by loading up the dataset to get more familiar with it.

In [122]:
import helper

codes = helper.load_data('cipher.txt')
plaintext = helper.load_data('plaintext.txt')

Now `codes` and `plaintext` are both arrays with each element being a phrase. The first three encoded phrases are:

In [123]:
codes[:5]

['YMJ QNRJ NX MJW QJFXY QNPJI KWZNY , GZY YMJ GFSFSF NX RD QJFXY QNPJI .',
 'MJ XFB F TQI DJQQTB YWZHP .',
 'NSINF NX WFNSD IZWNSL OZSJ , FSI NY NX XTRJYNRJX BFWR NS STAJRGJW .',
 'YMFY HFY BFX RD RTXY QTAJI FSNRFQ .',
 'MJ INXQNPJX LWFUJKWZNY , QNRJX , FSI QJRTSX .']

And their plaintext versions are:

In [124]:
plaintext[:5]

['THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .',
 'HE SAW A OLD YELLOW TRUCK .',
 'INDIA IS RAINY DURING JUNE , AND IT IS SOMETIMES WARM IN NOVEMBER .',
 'THAT CAT WAS MY MOST LOVED ANIMAL .',
 'HE DISLIKES GRAPEFRUIT , LIMES , AND LEMONS .']

In [125]:
len(codes)

10001

In [126]:
len(plaintext)

10001

In [136]:
max([len(sentence) for sentence in codes])

101

## Model Overview: Character-Level RNN
The model we will use here is a character-level RNN since the cipher seems to work on the characer level. In a machine translation scenario, a word-level RNN is the more common choice.

A character-level RNN will take as input an integer referring to a specific character and output another integer. To be able to get our model to work, we'll need to preprocess our dataset in the following steps:
 1. Isolating each character as an array element (instead of an entire phrase, or word being the element of the array)
 1. Tokenizing the characters so we can turn them from letters to integers and vice-versa
 1. Padding the strings so that all the inputs and outputs can fit in matrix form
 
To visualize this processing, let's assume either our source sequences (`codes` in this case) or target sequences (`plaintext` in this case) look like this (a list of strings):

<img src="list_1.png" />

Since this model will be working on the character level, we'll need to separate each string into a list of characters (implicitly done by the tokenizer in this notebook):

<img src="list_2.png" />

Then, the process of tokenization will turn each character into an integer.  Note that when you're working on the a word-level RNN (as in most machine translation examples), the tokenizer will assign an integer to each word rather than each letter, and each cell would represent a word rather than a character.

<img src="list_3.png" />

Most machine learning platforms expect the input to be a matrix rather than a list of lists. To turn the input into a matrix, we need to find the longest member of the list, and pad all shorter sequences with 0. Assuming 'and two' is the longest sequence in this example, the matrix ends up looking like this:

<img src="padded_list.png" />
 
## Preprocessing (IMPLEMENT)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Since we're working on the character level, make sure to set the `char_level` flag to the appropriate value. Then, fit the tokenizer on x.

In [127]:
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    
    tf.keras.preprocessing.text.Tokenizer(num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=" ",
    char_level=False,
    oov_token=None,
    document_count=0,
    **kwargs
    )
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level=True)
    x_tk.fit_on_texts(x)                 # because input is text, not sequence (list of integer tokens)

    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{' ': 1, 'e': 2, 'o': 3, 'i': 4, 's': 5, 'h': 6, 'r': 7, 'y': 8, 'u': 9, 'c': 10, 'n': 11, 't': 12, 'a': 13, 'p': 14, '.': 15, 'T': 16, 'q': 17, 'k': 18, 'w': 19, 'f': 20, 'x': 21, 'm': 22, 'v': 23, 'l': 24, 'z': 25, 'd': 26, 'g': 27, 'b': 28, 'j': 29, 'B': 30, 'J': 31, ',': 32}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [16, 6, 2, 1, 17, 9, 4, 10, 18, 1, 28, 7, 3, 19, 11, 1, 20, 3, 21, 1, 29, 9, 22, 14, 5, 1, 3, 23, 2, 7, 1, 12, 6, 2, 1, 24, 13, 25, 8, 1, 26, 3, 27, 1, 15]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [30, 8, 1, 31, 3, 23, 2, 1, 32, 1, 22, 8, 1, 17, 9, 4, 10, 18, 1, 5, 12, 9, 26, 8, 1, 3, 20, 1, 24, 2, 21, 4, 10, 3, 27, 7, 13, 14, 6, 8, 1, 19, 3, 11, 1, 13, 1, 14, 7, 4, 25, 2, 1, 15]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [16, 6, 4, 5, 1, 4, 5, 1, 13, 1, 5, 6, 3, 7, 12, 1, 5, 2, 11, 12, 2, 11, 10, 2, 1, 15]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the cipher sequences have the same length and all the plaintext sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [128]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    
    tf.keras.preprocessing.sequence.pad_sequences(
    sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre',
    value=0.0)
    """
    # TODO: Implement
    # Find the length of the longest string in the dataset.
    if length is None:
        length = max([len(sentence) for sentence in x])
    # Then, pass it to pad_sentences as the maxlen parameter
    
    return pad_sequences(x, maxlen=length, padding="post", truncating="post",)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [16  6  2  1 17  9  4 10 18  1 28  7  3 19 11  1 20  3 21  1 29  9 22 14  5
  1  3 23  2  7  1 12  6  2  1 24 13 25  8  1 26  3 27  1 15]
  Output: [16  6  2  1 17  9  4 10 18  1 28  7  3 19 11  1 20  3 21  1 29  9 22 14  5
  1  3 23  2  7  1 12  6  2  1 24 13 25  8  1 26  3 27  1 15  0  0  0  0  0
  0  0  0  0]
Sequence 2 in x
  Input:  [30  8  1 31  3 23  2  1 32  1 22  8  1 17  9  4 10 18  1  5 12  9 26  8  1
  3 20  1 24  2 21  4 10  3 27  7 13 14  6  8  1 19  3 11  1 13  1 14  7  4
 25  2  1 15]
  Output: [30  8  1 31  3 23  2  1 32  1 22  8  1 17  9  4 10 18  1  5 12  9 26  8  1
  3 20  1 24  2 21  4 10  3 27  7 13 14  6  8  1 19  3 11  1 13  1 14  7  4
 25  2  1 15]
Sequence 3 in x
  Input:  [16  6  4  5  1  4  5  1 13  1  5  6  3  7 12  1  5  2 11 12  2 11 10  2  1
 15]
  Output: [16  6  4  5  1  4  5  1 13  1  5  6  3  7 12  1  5  2 11 12  2 11 10  2  1
 15  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [129]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_code_sentences, preproc_plaintext_sentences, code_tokenizer, plaintext_tokenizer =\
    preprocess(codes, plaintext)

print('Data Preprocessed')

Data Preprocessed


In [130]:
preproc_code_sentences[0]

array([ 5, 14,  3,  1, 10,  2, 13,  3,  1,  2,  4,  1, 14,  3,  6,  1, 10,
        3,  8,  4,  5,  1, 10,  2, 25,  3, 11,  1, 20,  6,  9,  2,  5,  1,
       18,  1, 17,  9,  5,  1,  5, 14,  3,  1, 17,  8,  7,  8,  7,  8,  1,
        2,  4,  1, 13, 15,  1, 10,  3,  8,  4,  5,  1, 10,  2, 25,  3, 11,
        1, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [131]:
len(code_tokenizer.word_index)+1  # +1 for 0?

32

In [132]:
len(plaintext_tokenizer.word_index)+1

32

In [133]:
plaintext_tokenizer.word_index

{' ': 1,
 'I': 2,
 'E': 3,
 'S': 4,
 'T': 5,
 'R': 6,
 'N': 7,
 'A': 8,
 'U': 9,
 'L': 10,
 'D': 11,
 'O': 12,
 'M': 13,
 'H': 14,
 'Y': 15,
 'G': 16,
 'B': 17,
 ',': 18,
 '.': 19,
 'F': 20,
 'P': 21,
 'C': 22,
 'V': 23,
 'W': 24,
 'K': 25,
 'J': 26,
 'X': 27,
 'Q': 28,
 'Z': 29,
 '?': 30,
 "'": 31}

By default, the output of a RNN layer contains a single vector per sample. This vector is the RNN cell output corresponding to the last timestep, containing information about the entire input sequence. The shape of this output is (batch_size, units) where units corresponds to the units argument passed to the layer's constructor.

A RNN layer can also return the entire sequence of outputs for each sample (one vector per timestep per sample), if you set return_sequences=True. The shape of this output is (batch_size, timesteps, units).

In addition, a RNN layer can return its final internal state(s). The returned states can be used to resume the RNN execution later, or to initialize another RNN. This setting is commonly used in the encoder-decoder sequence-to-sequence model, where the encoder final state is used as the initial state of the decoder.

To configure a RNN layer to return its internal state, set the return_state parameter to True when creating the layer. Note that LSTM has 2 state tensors, but GRU only has one.

To configure the initial state of the layer, just call the layer with additional keyword argument initial_state. Note that the shape of the state needs to match the unit size of the layer

Return states in addition to output:
- output, state_h, state_c = layers.LSTM(64, return_state=True, name="encoder")(x)
- encoder_state = [state_h, state_c]

In [134]:
preproc_code_sentences.shape

(10001, 101)

In [135]:
preproc_plaintext_sentences.shape

(10001, 101, 1)

# LSTM

In [137]:
from keras.layers import GRU, Input, Dense, TimeDistributed, RNN, LSTM
from keras.models import Model, Sequential
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow


def simple_model(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    
    x = Input(shape=input_shape[1:])   # shape(none,101,1) ie 
    
    seq = LSTM(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    
    model = Model(inputs = x, outputs = output)
    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    
    model.summary()
    
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_code_sentences, preproc_plaintext_sentences.shape[1]) # pad code sequences with maxlen 54: shape=10001x54
tmp_x = tmp_x.reshape((-1, preproc_plaintext_sentences.shape[-2], 1))     # reshape padded code seq in 10001 x 54 x 1


In [138]:
tmp_x.shape

(10001, 101, 1)

In [149]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 101, 1)            0         
_________________________________________________________________
Layer1 (LSTM)                (None, 101, 64)           16896     
_________________________________________________________________
time_distributed_18 (TimeDis (None, 101, 32)           2080      
Total params: 18,976
Trainable params: 18,976
Non-trainable params: 0
_________________________________________________________________


In [150]:
simple_rnn_model.get_layer(name="Layer1").output.shape

TensorShape([Dimension(None), Dimension(None), Dimension(64)])

In [153]:
simple_rnn_model.fit(tmp_x, preproc_plaintext_sentences, batch_size=32, epochs=4, validation_split=0.2)

Train on 8000 samples, validate on 2001 samples
Epoch 1/4
8000/8000 [==============================] - 36s 5ms/step - loss: 0.3267 - acc: 0.9262 - val_loss: 0.2858 - val_acc: 0.9362
Epoch 2/4
8000/8000 [==============================] - 36s 5ms/step - loss: 0.2529 - acc: 0.9448 - val_loss: 0.2237 - val_acc: 0.9532
Epoch 3/4
8000/8000 [==============================] - 36s 5ms/step - loss: 0.2005 - acc: 0.9575 - val_loss: 0.1791 - val_acc: 0.9611
Epoch 4/4
8000/8000 [==============================] - 36s 4ms/step - loss: 0.1613 - acc: 0.9665 - val_loss: 0.1451 - val_acc: 0.9706


In [154]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], plaintext_tokenizer))

`logits_to_text` function loaded.
T H E   L I M E   I S   H E R   L E A S T   L I K E D   F R U I T   ,   B U T   T H E   G A N A N A   I S   M O   L E A S T   L I K E D   . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [89]:
plaintext[0]

'THE LIME IS HER LEAST LIKED FRUIT , BUT THE BANANA IS MY LEAST LIKED .'

And there it is. The RNN was able to learn this basic character-level cipher (which was a simple [Caesar cipher](https://en.wikipedia.org/wiki/Caesar_cipher). If you want a bigger cryptography challenge, check out [Learning the Enigma with Recurrent Neural Networks](https://greydanus.github.io/2017/01/07/enigma-rnn/). 

# GRU

In [155]:
def simple_model1(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    
    x = Input(shape=input_shape[1:])   # shape(none,54,1) ie 
    
    seq = GRU(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    
    model = Model(inputs = x, outputs = output)
    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    
    model.summary()
    
    return model


In [156]:
# Train the neural network
simple_rnn_model1 = simple_model1(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 101, 1)            0         
_________________________________________________________________
Layer1 (GRU)                 (None, 101, 64)           12672     
_________________________________________________________________
time_distributed_19 (TimeDis (None, 101, 32)           2080      
Total params: 14,752
Trainable params: 14,752
Non-trainable params: 0
_________________________________________________________________


In [157]:
simple_rnn_model1.fit(tmp_x, preproc_plaintext_sentences, batch_size=128, epochs=16, validation_split=0.2)

Train on 8000 samples, validate on 2001 samples
Epoch 1/16
8000/8000 [==============================] - 21s 3ms/step - loss: 2.5627 - acc: 0.4277 - val_loss: 1.6755 - val_acc: 0.4654
Epoch 2/16
8000/8000 [==============================] - 20s 2ms/step - loss: 1.4602 - acc: 0.5440 - val_loss: 1.2939 - val_acc: 0.6118
Epoch 3/16
8000/8000 [==============================] - 20s 2ms/step - loss: 1.1693 - acc: 0.6538 - val_loss: 1.0674 - val_acc: 0.6844
Epoch 4/16
8000/8000 [==============================] - 20s 2ms/step - loss: 0.9792 - acc: 0.7207 - val_loss: 0.9052 - val_acc: 0.7441
Epoch 5/16
8000/8000 [==============================] - 20s 2ms/step - loss: 0.8352 - acc: 0.7654 - val_loss: 0.7752 - val_acc: 0.7864
Epoch 6/16
8000/8000 [==============================] - 20s 2ms/step - loss: 0.7156 - acc: 0.8070 - val_loss: 0.6637 - val_acc: 0.8269
Epoch 7/16
8000/8000 [==============================] - 20s 2ms/step - loss: 0.6139 - acc: 0.8448 - val_loss: 0.5714 - val_acc: 0.8597
Epoch 8

In [158]:
print(logits_to_text(simple_rnn_model1.predict(tmp_x[:1])[0], plaintext_tokenizer))

T H E   L I M E   I S   H E R   L E A S T   L I K E D   F R U I T   ,   B U T   T H E   B A N A N A   I S   M H   L E A S T   L I K E D   . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# SimpleRNN

In [165]:
from keras.layers import SimpleRNN 
def simple_model2(input_shape, output_sequence_length, code_vocab_size, plaintext_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param code_vocab_size: Number of unique code characters in the dataset
    :param plaintext_vocab_size: Number of unique plaintext characters in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the model
    
    x = Input(shape=input_shape[1:])   # shape(none,54,1) ie 
    
    seq = SimpleRNN(units= 64, return_sequences = True, activation="tanh", name='Layer1')(x)  # output must be batchsize x timesteps x units
    
    output = TimeDistributed(Dense(units = plaintext_vocab_size, activation='softmax', name='Layer2'))(seq)
    
    model = Model(inputs = x, outputs = output)
    
    model.compile(optimizer='adam', loss= sparse_categorical_crossentropy, metrics=['accuracy'])
    
    model.summary()
    
    return model


In [166]:
# Train the neural network
simple_rnn_model2 = simple_model2(
    tmp_x.shape,
    preproc_plaintext_sentences.shape[1],
    len(code_tokenizer.word_index)+1,
    len(plaintext_tokenizer.word_index)+1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 101, 1)            0         
_________________________________________________________________
Layer1 (SimpleRNN)           (None, 101, 64)           4224      
_________________________________________________________________
time_distributed_21 (TimeDis (None, 101, 32)           2080      
Total params: 6,304
Trainable params: 6,304
Non-trainable params: 0
_________________________________________________________________


In [167]:
simple_rnn_model2.fit(tmp_x, preproc_plaintext_sentences, batch_size=128, epochs=16, validation_split=0.2)

Train on 8000 samples, validate on 2001 samples
Epoch 1/16
8000/8000 [==============================] - 12s 2ms/step - loss: 2.1848 - acc: 0.4288 - val_loss: 1.5287 - val_acc: 0.5425
Epoch 2/16
8000/8000 [==============================] - 11s 1ms/step - loss: 1.3699 - acc: 0.5847 - val_loss: 1.2494 - val_acc: 0.6178
Epoch 3/16
8000/8000 [==============================] - 11s 1ms/step - loss: 1.1558 - acc: 0.6431 - val_loss: 1.0809 - val_acc: 0.6626
Epoch 4/16
8000/8000 [==============================] - 11s 1ms/step - loss: 1.0124 - acc: 0.6913 - val_loss: 0.9570 - val_acc: 0.7188
Epoch 5/16
8000/8000 [==============================] - 11s 1ms/step - loss: 0.9021 - acc: 0.7386 - val_loss: 0.8567 - val_acc: 0.7708
Epoch 6/16
8000/8000 [==============================] - 11s 1ms/step - loss: 0.8102 - acc: 0.7834 - val_loss: 0.7715 - val_acc: 0.7980
Epoch 7/16
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7315 - acc: 0.8086 - val_loss: 0.6982 - val_acc: 0.8148
Epoch 8

In [168]:
print(logits_to_text(simple_rnn_model2.predict(tmp_x[:1])[0], plaintext_tokenizer))

T H E   L I M E   I S   M E R   L E A S T   L I K E D   F R U I T   ,   B U T   T H E   . A N A N A   I S   M H   D E A S T   L I K E D   . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


Overall:
- the GRU has the least errors (only 1) but required higher training time with 16 epochs for 128 batch size
- SimpleRNN with the same training has much more errors
- Finally LSTM has 2 errors only and was trained with only 4 epochs and 64 batch size which is twice as less as the GRU

Conclusion: best performance for LSTM with GRU next